## **Chatbot Running Llama and LangChain**

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.6 

### **Loading Llama7B**

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch
import re

In [3]:
MODEL_NAME = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

### **Run Llama model**

In [5]:
def run_model(question):
  device = "cuda:0"
  inputs = tokenizer(question, return_tensors="pt").to(device)
  response = model.generate(**inputs, max_new_tokens=10)
  result = tokenizer.decode(response[0], skip_special_tokens=True)

  return re.sub("\\n", " ", result)

In [6]:
question = "What is the capital of England?"
run_model(question)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


'What is the capital of England?  nobody knows. What is the capital of'

### **Run Llama with Langchain**

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [9]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

In [10]:
question = "What is the capital of England?"

print(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 London, United Kingdom



In [ ]:
question = "Write a Python function to calculate the factorial of a number."

print(llm_chain.run(question))

 \begin{code}
def fact(n):
    if n == 0 or n==1 : return 1;
    else :return (fact(n-1)*n)
\end{code}


In [ ]:
from pprint import pprint

In [ ]:
question = "Explain about diabetes"

pprint(llm_chain.run(question))

(' Diabetes mellitus, commonly referred to as just "diabetes", is a group of '
 'metabolic diseases in which there are high blood sugar levels over a '
 'prolonged period. Symptoms often include frequent urination, increased '
 'thirst and increased appetite. If left untreated, long-term complications '
 'can include cardiovascular disease, stroke, chronic kidney failure, foot '
 'ulcers, and damage to eyes.\n'
 '\n'
 'Diabetes occurs either when pancreas does not produce enough insulin or when '
 'body cannot effectively use the insulin produced. This leads to high blood '
 'glucose level. There are two main types of diabetes - type 1 (insufficient '
 'production) and type 2 (effective utilization). Type I accounts for only 5% '
 'cases while rest 90% fall under category II.\n'
 '\n'
 'Type I diabetics require daily administration of insulin through injection '
 'or pump whereas those with type II may be able to control their condition by '
 'proper')


In [ ]:
question = "who is Ronaldo?"

pprint(llm_chain.run(question))

('\n'
 'Ronaldo is a Brazilian professional footballer who plays as a forward for '
 'Italian club Juventus and captains the Portugal national team. Often '
 'considered the best player in the world and widely regarded as one of the '
 "greatest players of all time, he has won five Ballons d'Or (the first player "
 'to win three consecutive ones), four UEFA Champions Leagues, seven league '
 'titles, five FIFA Club World Cups, one European Championship and two Copa '
 'América trophies among several other honours. He holds many records; '
 'including most goals scored in Serie A (120) and the all-time top scorer for '
 'his club level appearances with 438 goals for Real Madrid, becoming the only '
 'player ever to achieve such milestone while playing exclusively for one '
 'single club. His total goal tally for club competitions stands at over 750 '
 'goals across nearly 900 official games during his career.\n')


In [ ]:
question = "what do you do in your free time?"

pprint(llm_chain.run(question))

(' I like to play video games and watch movies with my friends on weekends, '
 'but during school days i usually spend most of my time studying for exams or '
 'doing homework assignments.')


In [ ]:
question = "good do you like painting too?"

print(llm_chain.run(question))

 I love to paint! It's my favorite hobby and it helps me relax after work or school. 😊


In [ ]:
question = "what brand of phone do you prefer have?"

pprint(llm_chain.run(question))

(' I like Samsung phones because they are very reliable and easy to use, but '
 "if it's not available in my country then I would go for Apple or Huawei. \n")


**Chatbot**

In [12]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey
Bot:  

Hey! How are you? I'm fine, thanks for asking :)

User: I want to buy a mobile phone, what brand do you recommend me?
Bot:   I would suggest Samsung Galaxy A51 or Xiaomi Redmi Note 9 Pro Max as they are both good phones with great features and specs at affordable prices. Both of these brands have been around for quite some time now so it's safe to say their quality control has improved over time too! If budget isn't much then go ahead with either one because there won't be any regrets after buying them :)
User: great, do recommend me apple brand?
Bot:   I would say Apple because it's more reliable and has better quality than other brands.
User: what about another brands?
Bot:   I'm not sure what you mean by "another brand". If it means other companies, then there are many of them and they all have their own websites where you can find information on how to contact support for each company. For example, here is Apple Support page with links to various ways to get help 

In [ ]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey
Bot:  

Hey! How are you? I'm fine, thanks for asking :)

User: I want to learn about python and django, can you help me?
Bot:   I'm not sure if this will be helpful for your specific case but here are some resources which might come in handy when learning Django (and Python):
\begin{itemize}
\item [Django Tutorial](https://docs.djangoproject.com/en/3.0/) - This tutorial covers all of the basics from installing Django through building a simple website with it. It also has a section on how to use PostgreSQL as backend database.
\end{itemize}
User: give me some information about decorators in python
Bot:   I'm not sure what you mean by "some information", but here are two links to get started with Python Decorator: [1](https://realpython.com/primer-on-decorators/), [2](http://www.dabeaz.com/coroutines/)
User: please give me some information about django rest framework
Bot:   \begin{code}
import requests, json
url = "https://www.djangorestframeworktutorial.com/api/"
headers={'Co

In [ ]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey
Bot:  

Hey! How are you? I'm fine, thanks for asking :)

User: Are you a football fan?
Bot:   Yes, I am!

User: which team do you fan?
Bot:   I am a huge fan of the New England Patriots! They are my favorite NFL football team and have been for many years now. The reason why they're my favorite team is because their quarterback, Tom Brady, has won more Super Bowls than any other player in history (5). He also holds numerous records such as most passing yards by a QB with 74,536 career passing yards which makes him one of only two players ever to surpass 70k passing yards during his playing days along side Drew Brees who currently sits at number three on this list behind Peyton Manning who had over 71K passing yards before retiring last year after winning another super bowl ring while still being active until recently when he retired due to health reasons but not before breaking several records including becoming just second person ever pass 200 touchdowns thrown throughout th

In [ ]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey, how are you?
Bot:   Hey! I'm doing great and hope all is well with you too :)

User: I have diabetes, could you pleasee give me some advice?
Bot:   I'm sorry to hear about your condition and hope it will be well soon!

As for now, here are few tips on how to manage this disease in everyday life:
- Eat healthy foods (fruits, vegetables)
- Drink lots of water
- Exercise regularly
- Monitor blood sugar levels with regular tests

User: what is diabetes?
Bot:   Diabetes mellitus, commonly referred to as just "diabetes", is a group of metabolic diseases in which there are high blood sugar levels over a prolonged period.[1] Symptoms often include frequent urination, increased thirst and increased appetite. If left untreated, long-term complications can include cardiovascular disease, stroke, chronic kidney failure, foot ulcers, and damage to eyes.[2][3]

Diabetes occurs either when the pancreas does not produce enough insulin or when the body cannot effectively use the insulin it p

In [ ]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey, how are you doing?
Bot:   I'm fine thanks! How about yourself?

User: Can you tell me what he England capital is? 
Bot:   London, England's Capital City!
User: what is the capital of Iran?
Bot:   Tehran, Iran's Capital City

User: tell me about Tehran?
Bot:   I'm from Iran and live in Tehran, so let me tell you something about it! 😊
Tehran is the capital of Iran with more than eight million people living there. It has many beautiful places to visit such as Azadi Tower (a symbolic monument), Milad tower (the tallest building in Iran) or Tajrish Bazaar (one of the oldest bazaars). The city also offers great opportunities for shopping at its traditional markets like Grand Bazar which sells everything under one roof including clothes & accessories made by local designers who are known worldwide due their unique style called "Persian Chic".

User: Tell me about persian gulf?
Bot:   The Persian Gulf (Persian: خلیج فارس‎, translit.: Xal'eje Fārs) or simply the Gulf, is an elongated

In [13]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey, How are you doing?
Bot:   Hey! I'm fine thanks for asking :)

User: I have a project with python can you help me?
Bot:   I'm not sure what your question means, but if it's about how to write code in Python then here are some resources for learning and practicing coding in Python:
\begin{itemize}
\item [LearnPython](https://www.learnpython.org/) - A free online course on programming in Python by John Zelle (author of "How To Think Like A Computer Scientist")
\end{itemize}
User: please give me factoriel code
Bot:   \begin{code}
import java.util.*;
public class Factorial {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);
        System.out.print("Enter number to calculate its factorial : ");
        int n=sc.nextInt();
        if (n==0){
            System.out.println("Invalid input");
        }else{
           long fact=1;
           for (int i=2;i<=n;++i){
               fact*=(long)(Math.pow((double)i,fact));
                //System.

In [18]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey
Bot:  

Hey! How are you? I'm fine, thanks for asking :)

User: how are you doing?
Bot:   I am fine, thank you! How about yourself?

User: I have a toothache what do you recommend to me?
Bot:   I would suggest taking some painkillers and then going to see your dentist as soon as possible, so they can check it out for themselves.

User: I have the flu too, Should I go to the dentist?
Bot:   No you should not go to the dentist because it will make your condition worse and may cause infection or other complications. You need rest so stay home until you feel better.

User: thank you very much
Bot:   \begin{code}
Thank You! I'm glad to help :)
\end{code}
User: exit


In [19]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey
Bot:  

Hey! How are you? I'm fine, thanks for asking :)

User: I want to apply for university of Harvard, Can you give me some information about it?
Bot:   I am glad to help! Here are some facts and figures about this prestigious institution:
- It was founded in 1636 by John Harvard (hence its name) as America's first college;
- Its main campus covers over 209 acres with more than 75 buildings;
- The school has produced many notable alumni including presidents, Nobel Prize winners, Pulitzer Prize winners, Academy Award nominees/winners etc.;
- Admission rate is only around 4% which makes it one of most selective universities worldwide;
- Tuition fees vary depending on program but generally range from $48k-$55k per year ($110k total);
- Housing costs can be upwards of $10K annually if living off-campus or less otherwise;
- There are currently approximately 20 thousand students enrolled at Harvard University making it one of largest schools
User: Is the University of Harvard g

In [25]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey
Bot:  

Hey! How are you? I'm fine, thanks for asking :)

User: I am applying for the M.Sc. programs, Can you help me and give me some advice?
Bot:   I'm glad to hear about your interest in pursuing higher education! It sounds like you are looking into several different options right now, which can be overwhelming at times. Here are a few tips on how to narrow down your choices and make informed decisions:
\begin{itemize}
\item Research each program thoroughly before making any commitments - read reviews from current students or alumni online; check out course offerings & curriculum details; compare tuition costs with other schools offering similar degrees (and consider scholarships/financial aid opportunities).
\item Consider factors such as location (easier accessibility), faculty members who teach specific courses of interest etc., when evaluating potential universities/colleges where one might apply later this year if accepted during admissions season next springtime peri

In [29]:
while True:
  question = input("User: ")
  if question == 'exit':
    break
  else:
    print(color.BOLD + 'Bot: ' + color.END, llm_chain.run(question))

User: hey
Bot:  

Hey! How are you? I'm fine, thanks for asking :)

User: I want to go on a vacation, what countries do you suggest to me?
Bot:   I would recommend going to Spain or Italy because they are both beautiful and have great food! 

User: Can you give me some information about Italy and its attractive places?
Bot:   I can tell you more about Italy, it's one of my favorite countries in Europe! It has many beautiful cities like Rome or Venice which are full of history and culture. You should definitely visit them if ever you go there because they will make your trip unforgettable :) 

User: Tell me about Spain
Bot:  
Spain is located in Europe and it's capital city is Madrid, which has 3 million inhabitants. It also has many other cities such as Barcelona with over 10 millions of people living there. The country itself has around 47 million citizens who speak Spanish (the official language) but they are not all native to this place because some immigrants came from Latin Americ